In [63]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn.linear_model as skl
import re
import seaborn as sns
import json
import requests
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
import matplotlib.pyplot as plt
import boto3
import wget,time,os,requests,re
from multiprocessing import Pool, Queue, Process
import shutil as sh
from bs4 import BeautifulSoup
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
import selenium

In [64]:
past_values_path = '//*[@id="dnn_ctr377_View_tblValueHistoryRP"]'
base_url = "https://mcad-tx.org/Property-Detail/PropertyQuickRefID/"
RID = pd.DataFrame(['R363643','R380912','R447531','R474324','R474928','R490115','R496597'])
URL_list = pd.DataFrame({"RID":[base_url+x for x in RID[0]]})
url = URL_list["RID"][2]
html = requests.get(url).content
sel = Selector(text = html)

In [65]:
RID

,0
0,R363643
1,R380912
2,R447531
3,R474324
4,R474928
5,R490115
6,R496597


# New attempt at scraping MCAD datatable

In [66]:
def value_history_df(url):
    tables = pd.read_html(url,attrs={"id":"dnn_ctr377_View_tblValueHistoryRP"})
    table = tables[0][0][1]
    table_words = table.split()
    names = ["Year", "Improvement", "Land", "Market", "Ag Market", "Ag Use", "Appraised", "HS Cap Loss", "Assessed"]
    row_2020 = table_words[13:22]
    row_2019 = table_words[22:31]
    row_2018 = table_words[31:40]
    row_2017 = table_words[40:49]
    row_2016 = table_words[49:58]
    list_of_rows = [row_2020,row_2019,row_2018,row_2017,row_2016]
    df = pd.DataFrame(list_of_rows,columns=names)
    return df

In [67]:
def extract_land_size(xpath_land_size):
    land_path = '//*[@id="dnn_ctr377_View_tblLandSegmentsData"]'
    text = sel.xpath(land_path).extract()
    exp = "(\d*.\d*..acres)"
    cleaned = re.sub(",|\r|\t|\n","",text[0])
    matches = list(re.findall(exp,cleaned))
    return matches

In [68]:
def clean_imp_tables(url):
    html = requests.get(url).content
    sel = Selector(text = html)
    xpath_2021_Improvemetns = '//*[@id="tblCama"]//*[@class="fullWidthTable datatable"]'
    improvements_data = sel.xpath(xpath_2021_Improvemetns)

    total_improvements = []
    dfs = []

    improvements_data
    for i in range(0,len(improvements_data)):
        i+=1
        list_of_dat = pd.read_html(improvements_data[i-1].extract())
        list_of_imp = []
        rows_in_data = len(list_of_dat)
        names = list_of_dat[0].columns.values
        for row in list_of_dat[1:len(list_of_dat)]:
            if len(row)<2:
                list_of_imp.append(row.values[0])
            else:
                pass
        df = pd.DataFrame(list_of_imp,columns=names)
        df["Improvement #"] = i
        dfs.append(df)
    final_dfs = pd.concat(dfs)
    return final_dfs

In [69]:
def MCAD_PID_list_to_CSV(list_of_PID):
    

    
    # Path's to info on page
    
    land_path = '//*[@id="dnn_ctr377_View_tblLandSegmentsData"]'
    past_values_path = '//*[@id="dnn_ctr377_View_tblValueHistoryRP"]'
    owner_path = '//*[@id="dnn_ctr377_View_divOwnersLabel"]/text()'
    address_path = '//*[@id="dnn_ctr377_View_tdPropertyAddress"]/text()'
    

    # For every PID I will generate a URL and pull in the necessary info
    
    for PID in list_of_PID:
        PID_number = PID
        url = base_url+str(PID_number)
        
        
        
    # Getting page information
        
        html = requests.get(url).content
        sel = Selector(text = html)
        
        # Getting Land info
        text = sel.xpath(land_path).extract()
        exp = "(\d*.\d*..acres)"
        if len(text)<1:
            pass
        else:
            cleaned = re.sub(",|\r|\t|\n","",text[0])
            matches = list(re.findall(exp,cleaned))
        
        improvement_table = clean_imp_tables(url)
        df = value_history_df(url)
        df["Land Size"] = matches[0]
        df['Owner'] = sel.xpath(owner_path).extract()[0]
        df["Address"] = sel.xpath(address_path).extract()[0]
        df["MCAD PID"] = str(PID_number)
        
    # Write excel  sheet with multiple sheets
    
        writer = pd.ExcelWriter("{} spreadsheet.xlsx".format(PID_number), engine="xlsxwriter")
        
        df.to_excel(writer, sheet_name="Basic Info")
        improvement_table.to_excel(writer, sheet_name = "Improvements")
        
        writer.save()

# The BIG Test

In [70]:
def sum_brackets(a):
    return sum(a,[])

def retrieve_all_PIDs(PID_column):
    pid = PID_column.values
    vals = [x.split() for x in pid]
    group = set(sum_brackets(vals))
    pids = []
    for i in group:
        if len(i)>1:
            pids.append(i)
    return pids

In [61]:
comps_list = pd.read_excel("C:/Users/Erich Curtis/Downloads/0349 Subject & Comps Data.xlsx",sheet_name="PID List")
retrieve_all_PIDs(comps_list["PID's"])

['R242559',
 'R363657',
 'R402003',
 'R401991',
 'R298966',
 'R300564',
 'R363652',
 'R298969',
 'R351641']

In [72]:
MCAD_PID_list_to_CSV(retrieve_all_PIDs(comps_list["PID's"]))

In [71]:
pd.read_excel("C:/Users/Erich Curtis/Downloads/0349 Subject & Comps Data.xlsx")

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,SEARCH CRITERIA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,multi-tenant office w/parking garage,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,____ sf,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,built 1988-2008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,polygon search,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.12,NaN,NaN,NaN,NaN
6,NaN,NaN,MCAD,NaN,Property,Year,Eff.,NaN,Land,LTB,...,NaN,NaN,NaN,Q1 2020,NaN,Stabilized,NaN,NaN,Rent & Expenses,Lease
7,NaN,NaN,Account #,Address,Name,Built,Age,GBA,Size (ac),Ratio,...,NaN,Total,Value/sf,Occupancy,Gross Rent/sf,Vacancy/sf,Expenses/sf,Net Rent,Source/Notes,Type
8,NaN,Subject 1,R300564,"1450 LAKE ROBBINS DR, THE WOODLANDS, TX 77380",NaN,1998,11,142242,4.35298,1.33305,...,31.993223,9101560,63.9864,NaN,NaN,0,8,-8,"rent per CoStar, expenses per internal database",modified gross
9,NaN,Subject 2,R242559,"1330 LAKE ROBBINS DR, THE WOODLANDS, TX 77380",0,0,1010,0,0,NaN,...,NaN,0,NaN,0,0,0,8,-8,"rent per CoStar, expenses per internal database",full service
